In [ ]:
"""
!pip install requests
!pip install cherrypy
!pip install redis
!pip install paho-mqtt
"""

In [ ]:
import redis
import cherrypy
import json

# Redis connection setup
redis_host = 'redis-19881.c268.eu-west-1-2.ec2.cloud.redislabs.com'
redis_port = 19881
redis_password = 'Z3uevMG7GPbr5g66RbiGFWrQhsgKLdfO'
redis_timeseries_key = 'listening_to'

redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password)
try:
    is_connected = redis_client.ping()
    print('Redis client is connected ', is_connected)
except Exception as ex:
    print(ex)



class SensorData:
    @cherrypy.expose
    @cherrypy.tools.json_out()
    def sensors(self):
        # Retrieve all sensor data from Redis TimeSeries
        print("Received request on sensors endpoint")
        data = redis_client.execute_command('TS.RANGE', 'listening_to', '-', '+')
        probability = redis_client.execute_command('TS.RANGE', 'probability', '-', '+')
        response = {"values": []}
        for d, p in zip(data, probability):
            response["values"].append({"timestamp": d[0], "value": float(d[1]) * float(p[1])})

        # Return the data as a JSON response
        print(response)
        return json.dumps(response)

    @cherrypy.expose
    @cherrypy.tools.json_out()
    def sensor(self, id: str):
        # Retrieve sensor data for a specific ID from Redis TimeSeries
        # Retrieve all sensor data from Redis TimeSeries
        print("Received request on sensors endpoint")
        data = redis_client.execute_command('TS.RANGE', 'listening_to_' + id, '-', '+')
        probability = redis_client.execute_command('TS.RANGE', 'probability_' + id, '-', '+')
        response = {"values": []}
        for d, p in zip(data, probability):
            response["values"].append({"timestamp": d[0], "value": float(d[1]) * float(p[1])})

        # Return the data as a JSON response
        print(response)
        return json.dumps(response)

cherrypy.tree.mount(SensorData(), '/')

if __name__ == '__main__':
    cherrypy.config.update({
        'server.socket_host': '0.0.0.0',
        'server.socket_port': 8080
    })
    cherrypy.engine.start()

[12/Jul/2023:10:17:38] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[12/Jul/2023:10:17:38] ENGINE Started monitor thread 'Autoreloader'.
Redis client is connected  True
[b'probability_1', b'listening_to_enviroment', b'probability_s5', b'listening_to_environment', b'probability_enviroment', b'probability_environment', b'probability_s1', b'listening_to_s1', b'probability', b'listening_to_s5', b'probability_s2', b'listening_to_s3', b'probability_s4', b'listening_to', b'listening_to_s2', b'probability_s3', b'listening_to_s4']
[12/Jul/2023:10:17:38] ENGINE Serving on http://0.0.0.0:8080
[12/Jul/2023:10:17:38] ENGINE Bus STARTED


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1e190a0d-e0c4-4d69-ad9e-7a1f03e6c992' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>